In [1]:
import pandas as pd
import numpy as np

# Metodology


In [2]:
# Approach of making the healthcare data as the dependant variable are as follows
# 1. non essential columns from the dataset will be dropped
# 2. only variables date, High and Low will remain in the dataset
# 3. a new variable/column called avg_price will be made by taking the average of the high and low price
# 4. create a new variable/column named stock_trend
# 4. by comparing the avg_price of current date and the previous date, if the avg_price is higher, denote stock trend with 1,
#    else denote by 0.
# 5. the value 1 in the stock trend columns shows increase in avg_stock price, 0 shows decrease in stock price

# Importing healthcare data

In [3]:
hc_df = pd.read_csv('S&P 500 Health Care Historical Data.csv')
hc_df.head(3)

,Date,Price,Open,High,Low,Vol.,Change %
0,"Dec 09, 2020","1,306.48","1,311.95","1,316.01","1,300.26",240.14M,-0.29%
1,"Dec 08, 2020","1,310.26","1,300.40","1,311.57","1,298.92",234.52M,0.73%
2,"Dec 07, 2020","1,300.77","1,308.51","1,308.61","1,296.23",191.66M,-0.62%


# Inverting dataset due to reversed date order

In [4]:
hc_df = hc_df[::-1].reset_index(drop=True)
hc_df.head(5)

,Date,Price,Open,High,Low,Vol.,Change %
0,"Mar 24, 2020",937.08,906.88,938.93,904.89,348.87M,7.59%
1,"Mar 25, 2020",948.94,932.73,980.18,922.00,364.88M,1.27%
2,"Mar 26, 2020","1,015.18",957.61,"1,017.92",956.09,335.96M,6.98%
3,"Mar 27, 2020",990.69,995.14,"1,016.60",983.05,269.84M,-2.41%
4,"Mar 30, 2020","1,036.98","1,007.29","1,038.81","1,007.29",296.72M,4.67%


# Converting date to datetime format

In [5]:
hc_df['Date'] = pd.to_datetime(hc_df['Date'], format='%b %d, %Y')
hc_df.head(3)

,Date,Price,Open,High,Low,Vol.,Change %
0,2020-03-24,937.08,906.88,938.93,904.89,348.87M,7.59%
1,2020-03-25,948.94,932.73,980.18,922.00,364.88M,1.27%
2,2020-03-26,"1,015.18",957.61,"1,017.92",956.09,335.96M,6.98%


# Checking data types for dataset

In [6]:
hc_df.dtypes

Date        datetime64[ns]
Price               object
Open                object
High                object
Low                 object
Vol.                object
Change %            object
dtype: object

We need to change the high and low columns to float data type for the purpose of making a calculation later

# Changing data type and removal of non essential columns

In [7]:
#Dropping non essential columns
hc_df.drop(columns=['Price','Open','Vol.','Change %'], inplace=True)

#Removal of the , symbol in both High and Low columns
hc_df['High'] = hc_df['High'].str.replace(",", "")
hc_df['Low'] = hc_df['Low'].str.replace(",", "")

#Converting values in High and Low columns to float
hc_df['High']=hc_df['High'].astype(float)
hc_df['Low']=hc_df['Low'].astype(float)

# Creating new column (avg_price)

In [8]:
hc_df['avg_price'] = (hc_df['High'] + hc_df['Low'])/2

#High and Low column no longer needed, these columns will be dropped
hc_df.drop(columns=['High','Low'], inplace=True)
hc_df.head(11)

,Date,avg_price
0,2020-03-24,921.910
1,2020-03-25,951.090
2,2020-03-26,987.005
3,2020-03-27,999.825
4,2020-03-30,1023.050
5,2020-03-31,1032.920
6,2020-04-01,995.660
7,2020-04-02,1000.925
8,2020-04-03,1012.910
9,2020-04-06,1054.140


Note that the dates aren't continuous. Some dates are missing in between. Hence, we will first create a stock_trend column and denote all of them with the value 3. If the dates corresponding to the rows have a previous date ( strictly 1 day difference between the dates), compare the avg_price of the previous row and denote the trend column with either the value 1 or 0. After all processing is done, drop all rows that doesn't have a previous date ( value for stock_trend column is 3) and drop the avg_price column.

In [9]:
hc_df['stock_trend'] = 3

#temporarily convert date format to int (so that the difference can be calculated)
hc_df['Date'] = hc_df["Date"].dt.strftime("%Y%m%d")
hc_df['Date']=hc_df['Date'].astype(int)

for i in range (1,hc_df.shape[0]):
    day_diff = hc_df.loc[i].Date - hc_df.loc[i-1].Date
    
    if day_diff ==1 or day_diff>=70:
        if hc_df.loc[i].avg_price > hc_df.loc[i-1].avg_price:
            hc_df['stock_trend'][i] = 1
        
        elif hc_df.loc[i].avg_price < hc_df.loc[i-1].avg_price:
            hc_df['stock_trend'][i] = 0

<ipython-input-9-03770fbc4cb6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hc_df['stock_trend'][i] = 1
<ipython-input-9-03770fbc4cb6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hc_df['stock_trend'][i] = 0


In [10]:
hc_df.head(20)

,Date,avg_price,stock_trend
0,20200324,921.910,3
1,20200325,951.090,1
2,20200326,987.005,1
3,20200327,999.825,1
4,20200330,1023.050,3
5,20200331,1032.920,1
6,20200401,995.660,0
7,20200402,1000.925,1
8,20200403,1012.910,1
9,20200406,1054.140,3


In [11]:
hc_df = hc_df.loc[hc_df['stock_trend']!=3]
hc_df.drop(columns=['avg_price'], inplace=True)
hc_df.head(20)

,Date,stock_trend
1,20200325,1
2,20200326,1
3,20200327,1
5,20200331,1
6,20200401,0
7,20200402,1
8,20200403,1
10,20200407,1
11,20200408,1
12,20200409,1


# Reconverting date back to date time format

In [12]:
hc_df['Date'] = pd.to_datetime(hc_df['Date'], format= '%Y%m%d')
hc_df.head(3)

,Date,stock_trend
1,2020-03-25,1
2,2020-03-26,1
3,2020-03-27,1


In [13]:
hc_df.shape

(146, 2)

In [14]:
hc_df.to_csv('hc_df_preprocessed.csv', index=False)